# Initialization 

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import trim, col

# Read Bronze Table

In [0]:
df = spark.table("workspace.bronze.erp_loc_a101")

# Silver Transformation

## Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

## Customer ID Cleanup

In [0]:
df = df.withColumn("cid", F.regexp_replace("cid", "-", ""))